In [ ]:
!pip install diffusers transformers accelerate torch safetensors peft timm opencv-python

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Load txt2img pipe
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
txt2img_pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    )

# Load the LoRA weights
lora_weights_path = "/content/pytorch_lora_weights.safetensors"
txt2img_pipe.load_lora_weights(lora_weights_path)

txt2img_pipe.to("cuda")

In [ ]:
# Load IP-Adapter
txt2img_pipe.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin")

In [ ]:
scale = {
    "down": {"block_2": [0.0, 0.4]}, # Layout
    "up": {"block_0": [0.0, 0.85, 0.0]}, # Style
}
txt2img_pipe.set_ip_adapter_scale(scale)

In [ ]:
outfit = Image.open("/content/outfit_2.png")

In [ ]:
# Generate seed for reproducability
seed = 123  # You can choose any integer as the seed
generator = torch.manual_seed(seed)

In [ ]:
# Define the prompts
prompt = (
    "portrait of sks character looking to the right, sks face, white skin, sitting on log, moon and stars, "
    "zoomed out, neutral expression, looking at campfire, in the middle of a field with tall grass, detailed background, "
    "modern cartoon style, cartoon style, full-body, white t-shirt under jacket with no hood "
    "detailed character design, high quality background, good anatomy,"

)

negative_prompt = (
    "low-quality, blurry, deformed, disfigured, extra limbs, extra fingers,"
    "cropped, watermark, text, signature, nsfw, bad anatomy, unnatural proportions, "
    "distorted face, closed eyes, sad expression, outdated style, monochrome, "
    "overexposed, underexposed, grainy, pixelated, poor lighting, artifacts, "
    "duplicate, ugly, tiling, mutated hands, long neck, disconnected limbs, multiple characters, "
    "bad eyes, realistic, glasses, plain background, monochrome"
    )

In [ ]:
# Pass both the text and the IP adapter image to the pipeline
output_image = txt2img_pipe(prompt=prompt,
                            negative_prompt=negative_prompt,
                            ip_adapter_image=outfit,
                            generator=generator,
                            num_inference_steps=75,
                            num_images_per_prompt=3,
                            height=1024,
                            width=1024,
                            guidance_scale=10.5)#.images[0]

# Print and Save Images
for i, img in enumerate(output_image.images):
    img.save(f"generated-image-{i}.png")

    plt.imshow(img)
    plt.axis("off")
    plt.show()